<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastian Avendaño
- Nombre de alumno 2: Felipe Urrutia


### **Link de repositorio de GitHub:** https://github.com/furrutiav/lab-mds-2022

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

%load_ext autoreload
%autoreload 2

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [39]:
%%file bench_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting bench_module.py


In [4]:
votes, post, url = praw_reddit()

### Estudiar la función propuesta por el equipo docente.

La funcion $\texttt{praw_reddit}$ recibe un subreddit (por defecto "chile") y un numero denotado por $\texttt{n_hot}$ (por defecto 1000). Retorna un resumen de los reslutados entregados por la API *praw* de los top-post mas "hot" en el subreddit establecido en el input. En particular, entre los votos (up/down votes), post (titulo del post) y url (una ruta a cada post). Internamente la funtion determina los votos segun dos parametros de cada post (o submission). Esto son: upvote_ratio y score. El primero corresponde a la proporcion de votos up del total de votos. Mientras que el segundo es la diferencia entre los votos up y los votos down. Para determinar los votos up, basta determinar (upvote_ratio * score) / (2 * upvote_ratio - 1). Luego, los votos down, se determinan con votos up - score. El output final de la funcion es un trio (votes, post, url), donde cada elemento del trio es una lista de listas. Donde cada sublista es una version truncada de la lista que le sigue. Esto ya que internamente la funcion realiza un ciclo for con pasos de tamaño 50, y almacena los datos de los primeros k hot-post, con k multiplo positivo de 50. A priori, consideramos que esta ultima observacion produce que el algoritmo no sea optimo, al realizar iteraciones innecesarias.

### Estudiar los tiempos de ejecución del código a través de un perfilador.

In [11]:
!pip install line_profiler
%load_ext line_profiler

In [22]:
%lprun -f praw_reddit praw_reddit()

<img src="perfilador.jpg" width=900 />

**Respuesta:**

Del perfilador se concluye que la mayoria del tiempo (99.7% del tiempo de ejecucion de la funcion) es consumida por la linea 13. Esto corresponde al llamado subreddit.hot(limit=n_hot) con el objeto "subreddit" y metodo "hot". Este resultado es esperable dado que en la linea 2 se inicializa el wrapper (de la libreria praw) con las credenciales, lo que no consume tiempo. Pero, en la linea 13 se realiza la primera peticion a la API. Ademas, este tiempo puede ser grande ya que el parametro 𝚗⎯𝚑𝚘𝚝 tambien lo es. Ahora bien, si omitimos el tiempo que consume la linea 13, el otro gran porcentaje de tiempo utilizado se debe al ciclo for que comienza en la linea 18.

### Estudiar la memoria ocupada por el código a través de un perfilador.

In [29]:
!pip install memory_profiler
%load_ext memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=e572902a7ae98b599b7668791c1c83038454046fb925865f03451fef6de3fb15
  Stored in directory: c:\users\felip\appdata\local\pip\cache\wheels\01\ca\8b\b518dd2aef69635ad6fcab87069c9c52f355a2e9c5d4c02da9
Successfully built memory-profiler


In [40]:
from bench_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

<img src="perfilador_mem.jpg" width=900 />

**Respuesta:**

Del perfilador de memoria se concluye que la mayoria de la memoria reportada (13.9 MiB) es ocupada por la linea 13. Por misma razon que antes, esto corresponde al llamado de subreddit.hot(limit=n_hot), dado que recibe los resultados de la peticion realizada a la API. A diferencia de antes, el ciclo for no tiene una contribucion significativa en la memoria ocupada por la funcion.

### Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

**Respuesta:**

En resumen, tanto para el consumo de memoria como de tiempo es debido a la linea 13. Que corresponde a la peticion relizada a la API (tiempo) que devuelve los post mas "hot" (espacio). Tambien se menciono anteriormente que el ciclo for de la linea 18, consume otra gran parte del tiempo de ejecucion de la funcion. Lo utlimo, puede deberse a la gran cantidad de iteraciones que realizan las linea de codigo dentro del ciclo for (ver columna Hits en el reporte del perfilador de tiempo, o Ocurrences en el reporte del perfilador de memoria). Tambien notamos que la funcion tiene un try/except como medida de seguridad. Sin embargo, durante la ejecucion nunca se entra a la excepción.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

### Responder las preguntas

**Respuestas Teóricas:**

1.- [Memoria cache] La memoria cache es un tipo de memoria que nos permite almacenar informacion (datos). Este tipo de memoeria tiene el unico fin de acceder rapidamente en consultas posteriores. Dichas consultas (o accesos de memoria), tienen la caracteristica de ser repetitivas. Un ejemplo practico de su uso es en funcion recursivas, las cuales suelen ser optimizadas con programacion dinamica. 

1.- [LRU] Son las siglas en ingles de "Menos usado recientemente" (Least recently used). Este es un algoritmo basico para gestionar la memoria cache. La memoria de este tipo suele tener un largo pequeño, equivalente al numero de elementos almacenados. Si la memoria cache esta lleba, y se desea añadir un elemenot en la memoria, tal como lo dice el nombre del algoritmo, se descarta el elemento menos usado en recientemente en la memoria. Para esto, el algoritmo debe de mantener contadores para identificar los elementos menos usados (o consultados).

2.- Este tipo de algoritmos se utiliza para reducir el tiempo de ejecucion de una funcion (o codigo con inputs y output deterministico). La forma de lograr esto, es intercambiando tiempo por espacio. En este caso, el espacio es sinonimo de memoria. Tal como se motivo en la pregunta anterior, dicha memoria es de tipo cache. En efecto, los costos de utilizar técnicas de Caching son en memoria (o espacio).

3.- La consecuencía de ocupar caching en la función anterior es que al realizar otra vez el mismo llamado de esta funcion, se recibira el mismo output que quedo almacenado en memoria cache. Pese a que esto puede ser una buena idea, ya que el tiempo de ejecucion es considerable, utilizar este metodo no es del todo apropiado. Esto se debe a que la respuesta de la funcion depende del tiempo. Como se recibe hot-post, en un par se semanas hasta incluso dias, los post mas hot debieses de ser otros.

### Mejorar el código con cache.

In [92]:
@lru_cache
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

### Explicar las consecuencias de usar lru_cache sobre praw_reddit

Primer llamado:

In [93]:
%%timeit
praw_reddit(nombre_subreddit="chile", n_hot=1000)

The slowest run took 11.50 times longer than the fastest. This could mean that an intermediate result is being cached.
586 ns ± 702 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


Segundo llamado:

In [94]:
%%timeit
praw_reddit(nombre_subreddit="chile", n_hot=1000)

184 ns ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Se observa que el tiempo decrecion casi tres veces. [falta mas]

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [114]:
def fun1(botes):
    out = []
    for (u, d) in botes:
        a, b = 1+u, 1+d
        sgm = 1.65 * ( ( a*b / ( (a+b)**2 * (a+b+1))) ** 0.5 )
        mu = a / (a+b)
        out.append([mu, sgm])
    return out

In [127]:
def fun2(botes):
    a_b = botes+np.ones(1)
    a, b = a_b[:,0], a_b[:,1]
    sgm = 1.65 * np.sqrt( a*b / ((a+b)**2*(a+b+1)))
    mu = a/(a+b)
    out = np.column_stack((mu,sgm))
    return out

In [142]:
%%timeit
fun1(votes[9])

1.02 ms ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [143]:
%%timeit
fun2(votes[9])

17.9 µs ± 79.8 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [ ]:
###### Código Aquí ######

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [ ]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervals_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

In [ ]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

**Respuesta:**

`Escriba su respuesta aquí`

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>